# Import Functions and Libray

In [ ]:
import BAC0
import time
import os, sys
import paho.mqtt.client as mqtt
from random import randrange, uniform
import uuid

# Ignore this Code

In [ ]:
# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

# Function to Return BACnet Device Details for Connection

In [ ]:
def get_bac0_device(device, network):
    print("Get BAC0 device")
    name, vendor, address, device_id = device
    return BAC0.device(address, device_id, network, poll=0,
    object_list=None)

# Main function to connect with BACnet Device and Publish BACnet Device Data to Mosquitto MQTT Broker

In [ ]:
def main():
    # MQTT Setup
    
    # Local or azure mqtt broker address
    # mosquitto broker IP Address
    mqttBroker ="127.0.0.1"
    # incase of broker usename and password uncomment below line    
    # username = "mqtt_username"
    # password = "mqtt_password"
    
    # mosquito broker tcp port
    port = 1883
    
    # define client ID
    cid = str(uuid.uuid4())
    client = mqtt.Client("Gateway_Device_" + cid)
    
    # incase of broker usename and password uncomment below line
    # client.username_pw_set(username, password)
    
    #Connect to MQTT Broker
    client.connect(mqttBroker, port)

    # BAC0 Setup
    # Create BACnet Network
    bacnet = BAC0.connect()
    # discover all BACnet IP device from network
    bacnet.discover()
    # Use first device from list of discoverd devices
    device = bacnet.devices[0]
    # Fetch All info of BACnet Device like Device Name, Device Vendor Details , Device Address, Device ID     
    bac0_device = get_bac0_device(device, bacnet)
    
    # Main Loop of sending data to mosquitto mqtt broker
    while True:
        # Get list of points(Tags) with their latest values
        points = bac0_device.points

        # Ignor this code.
        blockPrint()
        print(points)
        enablePrint()
        
        for idx, point in enumerate(points):
            # point.properties.name > Will get you the BACnet Device Tag Value            
            # point.lastValue > Will get you the BACnet Device Tag Value          
            msg = str(point.lastValue)
            topic = "{}".format(point.properties.name)
            # publish data to mosquitto mqtt broker
            client.publish(topic, msg)
        
        # Wait for 4 seconds before sending updated data
        time.sleep(4)

In [ ]:
if __name__ == "__main__":
    # Run Main Function
    main()